# WDP sample data instances

## Preliminaries
### Formulations

In [7]:
using Pkg
Pkg.add("Combinatorics")
Pkg.add("SimpleWeightedGraphs")

using Combinatorics, Graphs, SimpleWeightedGraphs, DataStructures

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [3]:
# The number of items
k = 10

# The set of items
S = [i for i in 1:k]

# The number of bids to generate
l = 100

# Generate all combinations of S
all_combinations = [c for c in combinations(S)]

# Select l random combinations
random_subsets = rand(all_combinations, l)

# Bidding prices for each subset
prices = rand(1:100, l)

bids = Tuple{Array{Int64,1}, Int64}[]

for i in 1:l
    push!(bids, (random_subsets[i], prices[i]))
end

A = zeros(Int, k, l)

for j in 1:l
    for i in 1:k
        if i in random_subsets[j]
            A[i, j] = 1
        end
    end
end

### WDP-IP

In [4]:
using JuMP, Gurobi

m = Model(Gurobi.Optimizer)

@variable(m, x[1:l], Bin)

@objective(m, Max, sum(x[i] * prices[i] for i in 1:l))

for i in 1:k
    @constraint(m, sum(A[i, j]*x[j] for j in 1:l) <= 1)
end


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21


In [5]:
optimize!(m)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 10 rows, 100 columns and 500 nonzeros
Model fingerprint: 0xa4e840b2
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 224.0000000
Presolve removed 2 rows and 89 columns
Presolve time: 0.00s
Presolved: 8 rows, 11 columns, 37 nonzeros
Variable types: 0 continuous, 11 integer (11 binary)
Found heuristic solution: objective 266.0000000

Root relaxation: objective 3.070000e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [6]:
obj_value = objective_value(m)
sol = value.(x)

100-element Vector{Float64}:
  0.0
 -0.0
  0.0
  0.0
  0.0
  0.0
 -0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  1.0
  0.0
  0.0
  0.0

## Basic example
- 2 items 
- 3 bids

In [8]:
# Initial bids
bids = [(Set([1]),3), (Set([2]),4), (Set([1,2]),9), (Set([1,2]), 10), (Set([2]),1)]

function find_unique_bids(bids)
    # Using a dictionary for efficient look-up and update
    bid_dict = Dict{Set{Int}, Int}()

    for bid in bids
        subset, price = bid
        # If the subset is not in the dictionary or the new price is higher, update the dictionary
        if !haskey(bid_dict, subset) || bid_dict[subset] < price
            bid_dict[subset] = price
        end
    end

    # Convert the updated dictionary to a deque of tuples (for further operations if needed)
    unique_bids = Deque{Tuple{Set{Int}, Int}}()

    for (subset, price) in bid_dict
        push!(unique_bids, (subset, price))
    end

    return unique_bids
end

Updated bids in deque:
(Set([1]), 3)
(Set([2]), 4)
(Set([2, 1]), 10)


In [9]:
# bid 
S = [1, 2]
unique_bids = ((Set([1]),3),(Set([2]),4),(Set([1,2]),9))

((Set([1]), 3), (Set([2]), 4), (Set([2, 1]), 9))

In [34]:
n = length(unique_bids)
g = SimpleDiGraph(n+2)

{5, 0} directed simple Int64 graph

In [19]:
# SimpleWeightedDiGraph .ver
# sample
unique_bids = [(Set([1]),3), (Set([2]),4), (Set([1,2]),9)]

function create_graph(unique_bids)
    # Number of unique bids
    m = length(unique_bids)

    # Create a graph with m+2 nodes 
    n_nodes = m + 2
    g = SimpleWeightedDiGraph(n_nodes)

    # Define source and target node 
    source = n_nodes - 1
    target = n_nodes

    for i in 1:m
        add_edge!(g, source, i, unique_bids[i][2])
        add_edge!(g, i, target, 1)
    end

    for i in 1:m
        for j in 1:m
            if i != j && isempty(intersect(unique_bids[i][1], unique_bids[j][1]))
                add_edge!(g, i, j, unique_bids[j][2])
            end
        end
    end

    # subsets 
    subsets = [bid[1] for bid in unique_bids]

    return g, subsets
end

create_graph (generic function with 1 method)

In [20]:
g, subsets = create_graph(unique_bids)

({5, 8} directed simple Int64 graph with Float64 weights, Set{Int64}[Set([1]), Set([2]), Set([2, 1])])

In [22]:
subsets

3-element Vector{Set{Int64}}:
 Set([1])
 Set([2])
 Set([2, 1])

In [27]:
neighbors(g, 1)

2-element view(::Vector{Int64}, 1:2) with eltype Int64:
 2
 5

In [ ]:
# Longest path problem
function longestPath(s, adj, V)
    stack = Vector{Int}()
    visited = falses(V)
    dist = fill(-10^9, V)

    function topologicalSortUtil(v)
        visited[v] = true
        for i in adj[v]
            if !visited[i[1]]
                topologicalSortUtil(i[1])
            end
        end
        push!(stack, v)
    end

    # Topological Sort
    for i in 1:V
        if !visited[i]
            topologicalSortUtil(i)
        end
    end

    # Initialize distances to source
    dist[s] = 0

    # Process vertices in topological order
    while !isempty(stack)
        u = pop!(stack)

        if dist[u] != -10^9
            for i in adj[u]
                if dist[i[1]] < dist[u] + i[2]
                    dist[i[1]] = dist[u] + i[2]
                end
            end
        end
    end

    # Print calculated longest distances
    for i in 1:V
        if dist[i] == -10^9
            print("INF ")
        else
            print(dist[i], " ")
        end
    end
end

In [ ]:
function WDP_longest_path(g, subsets)
    n_nodes = nv(g)
    source = n_nodes - 1
    target = n_nodes
    dist = fill(-Inf, n_nodes)
    
    num_items = [maximum(subset) for subset in subsets]
    collected = falses(items)

    stack = Vector{Int}()
    visited = falses(n_nodes) 

    function dfs(v)
        visited[v] = true
        for u in neighbors(g, v)
            if !visited[u] || !collected[i for i in subset[u]]
                collected[i for i in subset[u]] = true
                dfs(u)
            end
        push!(stack, v)
        end
    end

    pred = fill(0, n_nodes)
    dist[source] = 0
    pred[source] = source

    for i in 1:n_nodes
        for e in edges(g)
            u, v = src(e), dst(e)
            if dist[u] + weight(e) > dist[v]
                dist[v] = dist[u] + weight(e)
                pred[v] = u
            end
        end
    end
    path = Int[]
    v = target
    while v != source
        pushfirst!(path, v)
        v = pred[v]
    end
    pushfirst!(path, source)
    return path
end

In [38]:
# sample
unique_bids = [(Set([1]),3), (Set([2]),4), (Set([1,2]),9)]

n = length(unique_bids) # Number of unique bids
g = SimpleDiGraph(n+2) # Graph with two additional nodes for source and target

# Define source and target node 
source = n + 1
target = n + 2

# Initialize a dictionary to store prices (capacities) of edges
prices = Dict{Tuple{Int, Int}, Int}()

# Add edges from source to all bid nodes with prices as their capacities
for i in 1:n
    add_edge!(g, source, i)
    prices[(source, i)] = unique_bids[i][2]
end

# Check each pair of bids to add edges if they are disjoint and set capacities
for i in 1:n
    for j in 1:n
        if i != j && isempty(intersect(unique_bids[i][1], unique_bids[j][1]))
            add_edge!(g, i, j)
            # Assuming you want to use the second bid's price for this edge's capacity
            prices[(i, j)] = unique_bids[j][2]
        end
    end
end

# Add edges from all bid nodes to the target node with 0 capacity (or some other logic)
for i in 1:n
    add_edge!(g, i, target)
    prices[(i, target)] = 0
end

In [39]:
collect(edges(g))

8-element Vector{Graphs.SimpleGraphs.SimpleEdge{Int64}}:
 Edge 1 => 2
 Edge 1 => 5
 Edge 2 => 1
 Edge 2 => 5
 Edge 3 => 5
 Edge 4 => 1
 Edge 4 => 2
 Edge 4 => 3

In [ ]:
maximum flow problem